# Final Project_Team 1

**Group Member:<br>`Lidan Cao`, `Yu Chen`, `Yexi Shen`, `Wentao Shi`**

Notation:
- **`DataFrame`**
- `Column`
- *`Data Type`*

## Import Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import numpy as np
import datetime

## Load Datasets

In [ ]:
locations = pd.read_csv('Locations-1.csv')
tanks = pd.read_csv('Tanks-1.csv')
invoices = pd.read_csv('Invoices-1.csv')
fuel_level_1 = pd.read_csv('Fuel_Level_Part_1-1.csv')
fuel_level_2 = pd.read_csv('Fuel_Level_Part_2-1.csv')

### Concatenate `fuel_level_1` and `fuel_level_2` as `fuel`
They have diffenrent column names, so we renamed the inconsistent column names in **`fuel_level_2`** before concat.

In [ ]:
fuel_level_2.rename(columns={'Fuel_Level':'Fuel Level','Timestamp':'Time stamp'},inplace=True) # column names to be consistent
fuel = pd.concat([fuel_level_1,fuel_level_2],ignore_index = True)

## Data Cleaning

### Missing Values Check in `locations`

In [ ]:
msno.matrix(locations)

There's no missing value in **`locations`**.

### Missing Values Check in `tanks`

In [ ]:
msno.matrix(tanks)

There's no missing value in **`tanks`**.

### Missing Values Check in `invoices`

DataFrame **`invoices`** contains locations other than 1-8, so we removed other locations' records.

In [ ]:
invoices = invoices[invoices['Invoice Gas Station Location']<=8] # remove locations other than 1-8
msno.matrix(invoices)

DataFrame **`invoices`** has missing values. We checked the records containing NaN before omitting them.

In [ ]:
invoices[invoices.isna().any(axis=1)]

Records with NaN are meaningless, can be omitted.

We calculated the proportion of the records containing NaN.

In [ ]:
invoices.isna().sum()

In [ ]:
print(36/len(invoices))
print(37/len(invoices)) # records can be omitted

The proportion is about 1%, can be omitted.

In [ ]:
invoices.dropna(inplace=True) # drop missing values records
invoices.reset_index(drop=True, inplace=True) # reset index
msno.matrix(invoices)

There's no missing value in **`invoices`** now.

In [ ]:
invoices.duplicated(['Invoice ID']).sum()

`Invoice ID` does not have duplicates.

### Missing Value Check in `fuel`

In [ ]:
msno.matrix(fuel)

DataFrame **`fuel`** has missing values. We checked the records containing NaN before omitting them.

In [ ]:
fuel[fuel.isna().any(axis=1)]

Records with NaN are meaningless, can be omitted.

We calculated the proportion of the records containing NaN.

In [ ]:
fuel.isna().sum()

In [ ]:
2/len(fuel) # records can be omitted

The proportion is tiny, can be omitted.

In [ ]:
fuel.dropna(inplace=True) # drop missing values records
fuel.reset_index(drop=True, inplace=True) # reset index
msno.matrix(fuel)

There's no missing value in **`fuel`** now.

In [ ]:
print("Duplicates:",fuel.duplicated(['Tank ID','Time stamp']).sum())
print("Proportion:",fuel.duplicated(['Tank ID','Time stamp']).sum()/len(fuel))

There are duplicates in **`fuel`** with a tiny proportion. We removed duplicates.

In [ ]:
fuel = fuel.drop_duplicates(['Tank ID','Time stamp'],keep=False) # drop all duplicates
fuel.reset_index(drop=True, inplace=True) # reset index
print("Duplicates:",fuel.duplicated(['Tank ID','Time stamp']).sum())

There's no duplicate in **`fuel`** now.

## Exploratory Data Analysis

Locaion information can be obtained from **`locations`**.

In [ ]:
locations.info()

In [ ]:
locations.head(8)

After looking at the table, we found that all gas station locations are in Ontario, Canada, except for location 8. We checked again and saw that location 8's latitude and longitude don't match a place in Canada. Instead, they point to a spot in the water between Washington DC and Virginia. Need to fix this error in the data for location 8.

DataFrame **`tanks_by_location`** provides information on the quantity and specific tanks at each location.

In [ ]:
tanks_by_location = tanks.groupby('Tank Location')['Tank ID'].apply(list).reset_index()
tanks_by_location.columns = ['Location', 'Tanks']
tanks_by_location = tanks_by_location.set_index('Location')
tanks_by_location['Total Tanks'] = tanks_by_location['Tanks'].apply(len)
tanks_by_location

### Cost

Cost information can be obtained from **`invoices`**.

In [ ]:
invoices.info()

The data type of `Invoice Date` was *`object`*, we converted it to *`datetime`* for future analysis.

In [ ]:
invoices['Invoice Date'] = pd.to_datetime(invoices['Invoice Date'])

#### Yearly Gross Purchase Cost

In [ ]:
invoices['Year'] = invoices['Invoice Date'].dt.year # add column 'Year' with value year from 'Invoice Date'
yearly_cost_by_location = invoices.groupby(['Year', 'Invoice Gas Station Location'])['Gross Purchase Cost'].sum() # calculate total cost for each location yearly
yearly_cost_by_location_unstacked = yearly_cost_by_location.unstack(fill_value=0) # transform the series into dataframe
# plot data using stacked bar chart
yearly_cost_by_location_unstacked.plot(kind='bar', stacked=True)
plt.title('Yearly Gross Purchase Cost')
plt.xlabel('Year')
plt.ylabel('Total Gross Purchase Cost ($)')
plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--')

In [ ]:
invoices['Invoice Date'].value_counts().sort_index()

- The chart uses a scientific notation (1e7 on the y-axis) indicating that the cost values are in the **tens of millions**.
- The total gross purchase cost for each year is a cumulative representation of all locations.
- **Location 1** consistently appears to have the highest gross purchase cost over the three years.
- The total gross purchase costs for 2017 and 2018 are almost the same, the cost for 2019 is about half that of the previous two years from the chart. However, from the raw data, we knew that the data for **2019 only covers January to August**, so it is reasonable that it is less than the previous years.

#### Monthly Gross Purchase Cost

In [ ]:
invoices['Year-Month'] = invoices['Invoice Date'].dt.to_period('M') # add column 'Year-Month' with value year-month from 'Invoice Date'
monthly_cost_by_location = invoices.groupby(['Year-Month', 'Invoice Gas Station Location'])['Gross Purchase Cost'].sum() # calculate total cost for each location
monthly_cost_by_location_unstacked = monthly_cost_by_location.unstack(fill_value=0) # transform the series into dataframe
# plot data using stacked bar chart
monthly_cost_by_location_unstacked.plot(kind='bar', stacked=True)
plt.title('Monthly Gross Purchase Cost')
plt.xlabel('Year-Month')
plt.ylabel('Total Gross Purchase Cost ($)')
plt.xticks(rotation=90)
plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--')

- The chart uses a scientific notation (1e6 on the y-axis) indicating that the cost values are in the **millions**.
- The total gross purchase cost for each month is a cumulative representation of all locations.
- **Location 1** consistently appears to have the highest gross purchase cost over monthes.
- The total gross purchase cost tends to be higher in **January** and **the middle of each year**.
- There is a pattern of fluctuation in total gross purchase costs over the months.
- The total gross purchase costs for 2019-08 is about half that of the previous month from the chart. However, from the raw data, we knew that the data for **2019-08 only covers up to the 15th**, so it is reasonable that it is less than the previous month.

#### Weekday Gross Purchase Cost

In [ ]:
invoices['Weekday'] = invoices['Invoice Date'].dt.dayofweek # add column 'Weekday' with value day-of-week from 'Invoice Date'
monthly_cost_by_location = invoices.groupby(['Weekday', 'Invoice Gas Station Location'])['Gross Purchase Cost'].sum() # calculate total cost for each location
monthly_cost_by_location_unstacked = monthly_cost_by_location.unstack(fill_value=0) # transform the series into dataframe
# plot data using stacked bar chart
monthly_cost_by_location_unstacked.plot(kind='bar', stacked=True)
plt.title('Weekday Gross Purchase Cost')
plt.xlabel('Weekday')
plt.ylabel('Total Gross Purchase Cost ($)')
weekday_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'] # transform numeric label into literal day-of-week
plt.xticks(ticks=range(0, 7), labels=weekday_labels)
plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--')

- The chart uses a scientific notation (1e7 on the y-axis) indicating that the cost values are in the **tens of millions**.
- The total gross purchase cost for each weekday is a cumulative representation of all locations.
- **Location 1** consistently appears to have the highest gross purchase cost over weekdays.
- **Tuesday** and **Friday**, show a higher total gross purchase cost compared to other days of the week.
- Besides weekdays, **Saturday** also has a few gross purchase cost.

### Fuel Level

Fuel level information can be obtained from **`fuel`**.

Perform a left outer join on **`fuel`** and **`tanks`** to associate fuel level records with corresponding locations as **`fuel_location`**.

In [ ]:
fuel_location = pd.merge(fuel, tanks, how='left', on='Tank ID') # left outer join fuel and tanks
fuel_location

In [ ]:
fuel_location.info()

The data type of `Time stamp` was *`object`*, we converted it to *`datetime`* for future analysis.

In [ ]:
fuel_location['Time stamp'] = pd.to_datetime(fuel_location['Time stamp'])

#### Average Fuel Level by Day of Week

In [ ]:
fuel_location['Day of Week'] = fuel_location['Time stamp'].dt.dayofweek # add column 'Day of Week' with value day-of-week from 'Time stamp'
average_weekday_fuel = fuel_location.groupby(['Tank Location', 'Day of Week'])['Fuel Level'].mean().reset_index() # calculate average fuel level
for loc_id in average_weekday_fuel['Tank Location'].unique():
    loc_data = average_weekday_fuel[average_weekday_fuel['Tank Location'] == loc_id] # get each location average data by day of week
    # plot each location's average fuel level by day of week
    plt.plot(loc_data['Day of Week'], loc_data['Fuel Level'], marker='o', linestyle='-', label=loc_id)
weekday_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'] # transform numeric label into literal day-of-week
plt.xticks(ticks=range(0, 7), labels=weekday_labels)
plt.xlabel('Day of the Week')
plt.ylabel('Average Fuel Level')
plt.title('Average Fuel Level by Day of Week for Each Location')
plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

#### Tank's Fuel Level Fluctuation

Grouped by **`Tank ID`** and with **`Time stamp`** sorted in ascending order, the difference in **`Fuel Level`** between each record and the previous one is the **`Fuel Level Change`** for each record. We determined the **`Fuel Level change`** for each tank.

##### Location 1

In [ ]:
fuel_Tank10 = fuel_location[fuel_location['Tank ID']=='T 10'] # get tank10 data
fuel_Tank10.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank10['Fuel Level Change'] = fuel_Tank10['Fuel Level'].diff() # calculate diffenrence
fuel_Tank10 = fuel_Tank10[fuel_Tank10['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank10['Time stamp'], fuel_Tank10['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 10')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank11 = fuel_location[fuel_location['Tank ID']=='T 11'] # get tank11 data
fuel_Tank11.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank11['Fuel Level Change'] = fuel_Tank11['Fuel Level'].diff() # calculate diffenrence
fuel_Tank11 = fuel_Tank11[fuel_Tank11['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank11['Time stamp'], fuel_Tank11['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 11')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank12 = fuel_location[fuel_location['Tank ID']=='T 12'] # get tank12 data
fuel_Tank12.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank12['Fuel Level Change'] = fuel_Tank12['Fuel Level'].diff() # calculate diffenrence
fuel_Tank12 = fuel_Tank12[fuel_Tank12['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank12['Time stamp'], fuel_Tank12['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 12')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank13 = fuel_location[fuel_location['Tank ID']=='T 13'] # get tank13 data
fuel_Tank13.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank13['Fuel Level Change'] = fuel_Tank13['Fuel Level'].diff() # calculate diffenrence
fuel_Tank13 = fuel_Tank13[fuel_Tank13['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank13['Time stamp'], fuel_Tank13['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 13')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank14 = fuel_location[fuel_location['Tank ID']=='T 14'] # get tank14 data
fuel_Tank14.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank14['Fuel Level Change'] = fuel_Tank14['Fuel Level'].diff() # calculate diffenrence
fuel_Tank14 = fuel_Tank14[fuel_Tank14['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank14['Time stamp'], fuel_Tank14['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 14')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank15 = fuel_location[fuel_location['Tank ID']=='T 15'] # get tank15 data
fuel_Tank15.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank15['Fuel Level Change'] = fuel_Tank15['Fuel Level'].diff() # calculate diffenrence
fuel_Tank15 = fuel_Tank15[fuel_Tank15['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank15['Time stamp'], fuel_Tank15['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 15')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

By examining the charts for **Tank 10** and **Tank 13**, we can see that the two are complementary, suggesting that they are the **same tank** but with different names. Tank 13's type is **P**, and Tank 10's type is **U**. P and U will be considered as **G** when order. Therefore, we considered to combine them together.

Since we dropped records with 0 `Fuel Level Change` to draw the plot before, we need the original records from **`fuel_location`** to keep the data consistence.

We merged the fuel level records of Tank 10 and Tank 13 into **`fuel_Tank10_Tank13`**. After sorting by Time stamp in ascendingly, we calculated the `Time Stamp Change` with the previous record. If the `Time Stamp Change` is less than 1 minute and the records come from different Tanks, we combine this two records into one by adding their `Fuel Level` values together, and change the `Tank Type` to 'G'. The `Tank ID` remains unchanged to indicate whether the change occurred in Tank 10 or Tank 13.

In [ ]:
# Concatenate records for Tanks 10 and 13
fuel_Tank10_2 = fuel_location[fuel_location['Tank ID'] == 'T 10'].copy()
fuel_Tank13_2 = fuel_location[fuel_location['Tank ID'] == 'T 13'].copy()
fuel_Tank10_Tank13 = pd.concat([fuel_Tank10_2, fuel_Tank13_2])

# Sort by 'Time stamp', calculate 'Fuel Level Change' and 'Time Stamp Change', and reset the index
fuel_Tank10_Tank13.sort_values(by=['Time stamp'], inplace=True)
fuel_Tank10_Tank13['Fuel Level Change'] = fuel_Tank10_Tank13['Fuel Level'].diff()
fuel_Tank10_Tank13['Time Stamp Change'] = fuel_Tank10_Tank13['Time stamp'].diff()
fuel_Tank10_Tank13.reset_index(drop=True, inplace=True)

# Set the drop threshold as 1 minute
time_threshold = pd.Timedelta('00:01:00')

# Identify rows to drop based on time threshold and tank ID change
mask = (fuel_Tank10_Tank13['Time Stamp Change'] <= time_threshold) & \
       (fuel_Tank10_Tank13['Tank ID'] != fuel_Tank10_Tank13['Tank ID'].shift())

# Update 'Fuel Level' and 'Tank Type' for the previous rows
fuel_Tank10_Tank13.loc[mask.shift(-1).fillna(False), 'Fuel Level'] += fuel_Tank10_Tank13.loc[mask, 'Fuel Level'].values
fuel_Tank10_Tank13.loc[mask.shift(-1).fillna(False), 'Tank Type'] = 'G'

# Drop the identified rows and recalculate 'Fuel Level Change'
fuel_Tank10_Tank13 = fuel_Tank10_Tank13[~mask]
fuel_Tank10_Tank13.reset_index(drop=True, inplace=True)
fuel_Tank10_Tank13['Fuel Level Change'] = fuel_Tank10_Tank13['Fuel Level'].diff()

# Drop rows with no fuel level change
fuel_Tank10_Tank13 = fuel_Tank10_Tank13[fuel_Tank10_Tank13['Fuel Level Change'] != 0]

fuel_Tank10_Tank13

In [ ]:
len(fuel_Tank10_Tank13[fuel_Tank10_Tank13['Fuel Level']>40000])/len(fuel_Tank10_Tank13)

There are 3.28% records' `Fuel Level` exceeds the capacity.

In [ ]:
fuel_Tank10_Tank13[fuel_Tank10_Tank13['Fuel Level']>40000]['Fuel Level'].describe()

By checking the describe, most of exceeding capacity records do not exceed 41000. Therefore, we removed these records within the margin of error.

In [ ]:
fuel_Tank10_Tank13 = fuel_Tank10_Tank13[fuel_Tank10_Tank13['Fuel Level']<=40000]

New plot combining Tank 10 and Tank 13 records.

In [ ]:
plt.plot(fuel_Tank10_Tank13['Time stamp'], fuel_Tank10_Tank13['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 10 and Tank 13')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

##### Location 2

In [ ]:
fuel_Tank16 = fuel_location[fuel_location['Tank ID']=='T 16'] # get tank16 data
fuel_Tank16.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank16['Fuel Level Change'] = fuel_Tank16['Fuel Level'].diff() # calculate diffenrence
fuel_Tank16 = fuel_Tank16[fuel_Tank16['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank16['Time stamp'], fuel_Tank16['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 16')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank17 = fuel_location[fuel_location['Tank ID']=='T 17'] # get tank17 data
fuel_Tank17.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank17['Fuel Level Change'] = fuel_Tank17['Fuel Level'].diff() # calculate diffenrence
fuel_Tank17 = fuel_Tank17[fuel_Tank17['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank17['Time stamp'], fuel_Tank17['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 17')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank18 = fuel_location[fuel_location['Tank ID']=='T 18'] # get tank18 data
fuel_Tank18.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank18['Fuel Level Change'] = fuel_Tank18['Fuel Level'].diff() # calculate diffenrence
fuel_Tank18 = fuel_Tank18[fuel_Tank18['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank18['Time stamp'], fuel_Tank18['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 18')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank19 = fuel_location[fuel_location['Tank ID']=='T 19'] # get tank19 data
fuel_Tank19.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank19['Fuel Level Change'] = fuel_Tank19['Fuel Level'].diff() # calculate diffenrence
fuel_Tank19 = fuel_Tank19[fuel_Tank19['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank19['Time stamp'], fuel_Tank19['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 19')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

##### Location 3

In [ ]:
fuel_Tank20 = fuel_location[fuel_location['Tank ID']=='T 20'] # get tank20 data
fuel_Tank20.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank20['Fuel Level Change'] = fuel_Tank20['Fuel Level'].diff() # calculate diffenrence
fuel_Tank20 = fuel_Tank20[fuel_Tank20['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank20['Time stamp'], fuel_Tank20['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 20')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank21 = fuel_location[fuel_location['Tank ID']=='T 21'] # get tank21 data
fuel_Tank21.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank21['Fuel Level Change'] = fuel_Tank21['Fuel Level'].diff() # calculate diffenrence
fuel_Tank21 = fuel_Tank21[fuel_Tank21['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank21['Time stamp'], fuel_Tank21['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 21')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

##### Location 4

In [ ]:
fuel_Tank22 = fuel_location[fuel_location['Tank ID']=='T 22'] # get tank22 data
fuel_Tank22.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank22['Fuel Level Change'] = fuel_Tank22['Fuel Level'].diff() # calculate diffenrence
fuel_Tank22 = fuel_Tank22[fuel_Tank22['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank22['Time stamp'], fuel_Tank22['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 22')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank23 = fuel_location[fuel_location['Tank ID']=='T 23'] # get tank23 data
fuel_Tank23.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank23['Fuel Level Change'] = fuel_Tank23['Fuel Level'].diff() # calculate diffenrence
fuel_Tank23 = fuel_Tank23[fuel_Tank23['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank23['Time stamp'], fuel_Tank23['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 23')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

##### Location 5

In [ ]:
fuel_Tank24 = fuel_location[fuel_location['Tank ID']=='T 24'] # get tank24 data
fuel_Tank24.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank24['Fuel Level Change'] = fuel_Tank24['Fuel Level'].diff() # calculate diffenrence
fuel_Tank24 = fuel_Tank24[fuel_Tank24['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank24['Time stamp'], fuel_Tank24['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 24')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank25 = fuel_location[fuel_location['Tank ID']=='T 25'] # get tank25 data
fuel_Tank25.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank25['Fuel Level Change'] = fuel_Tank25['Fuel Level'].diff() # calculate diffenrence
fuel_Tank25 = fuel_Tank25[fuel_Tank25['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank25['Time stamp'], fuel_Tank25['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 25')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

##### Location 6

In [ ]:
fuel_Tank26 = fuel_location[fuel_location['Tank ID']=='T 26'] # get tank26 data
fuel_Tank26.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank26['Fuel Level Change'] = fuel_Tank26['Fuel Level'].diff() # calculate diffenrence
fuel_Tank26 = fuel_Tank26[fuel_Tank26['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank26['Time stamp'], fuel_Tank26['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 26')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank27 = fuel_location[fuel_location['Tank ID']=='T 27'] # get tank27 data
fuel_Tank27.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank27['Fuel Level Change'] = fuel_Tank27['Fuel Level'].diff() # calculate diffenrence
fuel_Tank27 = fuel_Tank27[fuel_Tank27['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank27['Time stamp'], fuel_Tank27['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 27')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank28 = fuel_location[fuel_location['Tank ID']=='T 28'] # get tank28 data
fuel_Tank28.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank28['Fuel Level Change'] = fuel_Tank28['Fuel Level'].diff() # calculate diffenrence
fuel_Tank28 = fuel_Tank28[fuel_Tank28['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank28['Time stamp'], fuel_Tank28['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 28')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

##### Location 7

In [ ]:
fuel_Tank29 = fuel_location[fuel_location['Tank ID']=='T 29'] # get tank29 data
fuel_Tank29.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank29['Fuel Level Change'] = fuel_Tank29['Fuel Level'].diff() # calculate diffenrence
fuel_Tank29 = fuel_Tank29[fuel_Tank29['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank29['Time stamp'], fuel_Tank29['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 29')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank30 = fuel_location[fuel_location['Tank ID']=='T 30'] # get tank30 data
fuel_Tank30.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank30['Fuel Level Change'] = fuel_Tank30['Fuel Level'].diff() # calculate diffenrence
fuel_Tank30 = fuel_Tank30[fuel_Tank30['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank30['Time stamp'], fuel_Tank30['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 30')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

##### Location 8

In [ ]:
fuel_Tank31 = fuel_location[fuel_location['Tank ID']=='T 31'] # get tank31 data
fuel_Tank31.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank31['Fuel Level Change'] = fuel_Tank31['Fuel Level'].diff() # calculate diffenrence
fuel_Tank31 = fuel_Tank31[fuel_Tank31['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank31['Time stamp'], fuel_Tank31['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 31')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

In [ ]:
fuel_Tank32 = fuel_location[fuel_location['Tank ID']=='T 32'] # get tank32 data
fuel_Tank32.sort_values(by=['Time stamp'], inplace=True) # sort by 'Time stamp' ascendingly
fuel_Tank32['Fuel Level Change'] = fuel_Tank32['Fuel Level'].diff() # calculate diffenrence
fuel_Tank32 = fuel_Tank32[fuel_Tank32['Fuel Level Change']!=0] # drop no fuel level change records
# plot tank's fuel level fluctuation
plt.plot(fuel_Tank32['Time stamp'], fuel_Tank32['Fuel Level'], label='Fuel Level')
plt.title(f'Fuel Level for Tank 30')
plt.xlabel('Time')
plt.ylabel('Fuel Level')

### Fuel Consume

Each location's fuel consume can be obtained by **`fuel_Tank*`** where the `Fuel Level Change`<0.

We created **`fuel_consume_loc*`** for further analysis.

#### Location 1

In [ ]:
fuel_consume_loc1 = pd.concat([fuel_Tank10[fuel_Tank10['Fuel Level Change']<0],
                               fuel_Tank11[fuel_Tank11['Fuel Level Change']<0],
                               fuel_Tank12[fuel_Tank12['Fuel Level Change']<0],
                               fuel_Tank13[fuel_Tank13['Fuel Level Change']<0],
                               fuel_Tank14[fuel_Tank14['Fuel Level Change']<0],
                               fuel_Tank15[fuel_Tank15['Fuel Level Change']<0]],ignore_index=True)
fuel_consume_loc1

#### Location 2

In [ ]:
fuel_consume_loc2 = pd.concat([fuel_Tank16[fuel_Tank16['Fuel Level Change']<0],
                               fuel_Tank17[fuel_Tank17['Fuel Level Change']<0],
                               fuel_Tank18[fuel_Tank18['Fuel Level Change']<0],
                               fuel_Tank19[fuel_Tank19['Fuel Level Change']<0]],ignore_index=True)
fuel_consume_loc2

#### Location 3

In [ ]:
fuel_consume_loc3 = pd.concat([fuel_Tank20[fuel_Tank20['Fuel Level Change']<0],
                               fuel_Tank21[fuel_Tank21['Fuel Level Change']<0]],ignore_index=True)
fuel_consume_loc3

#### Location 4

In [ ]:
fuel_consume_loc4 = pd.concat([fuel_Tank22[fuel_Tank22['Fuel Level Change']<0],
                               fuel_Tank23[fuel_Tank23['Fuel Level Change']<0]],ignore_index=True)
fuel_consume_loc4

#### Location 5

In [ ]:
fuel_consume_loc5 = pd.concat([fuel_Tank24[fuel_Tank24['Fuel Level Change']<0],
                               fuel_Tank25[fuel_Tank25['Fuel Level Change']<0]],ignore_index=True)
fuel_consume_loc5

#### Location 6

In [ ]:
fuel_consume_loc6 = pd.concat([fuel_Tank26[fuel_Tank26['Fuel Level Change']<0],
                               fuel_Tank27[fuel_Tank27['Fuel Level Change']<0],
                               fuel_Tank28[fuel_Tank28['Fuel Level Change']<0]],ignore_index=True)
fuel_consume_loc6

#### Location 7

In [ ]:
fuel_consume_loc7 = pd.concat([fuel_Tank29[fuel_Tank29['Fuel Level Change']<0],
                               fuel_Tank30[fuel_Tank30['Fuel Level Change']<0]],ignore_index=True)
fuel_consume_loc7

#### Location 8

In [ ]:
fuel_consume_loc8 = pd.concat([fuel_Tank31[fuel_Tank31['Fuel Level Change']<0],
                               fuel_Tank32[fuel_Tank32['Fuel Level Change']<0]],ignore_index=True)
fuel_consume_loc8

### Fuel Order

In [ ]:
invoices['Amount Purchased'].describe()

#### Location 1

In [ ]:
fuel_Tank10_Tank13[fuel_Tank10_Tank13['Fuel Level Change']>0]

In [ ]:
fuel_Tank10_Tank13[fuel_Tank10_Tank13['Fuel Level Change']>0]['Fuel Level Change'].describe()

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_Tank10_Tank13[fuel_Tank10_Tank13['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
fuel_Tank10_Tank13

In [ ]:
fuel_order_loc1 = pd.concat([fuel_Tank10[fuel_Tank10['Fuel Level Change']>0],
                               fuel_Tank11[fuel_Tank11['Fuel Level Change']>0],
                               fuel_Tank12[fuel_Tank12['Fuel Level Change']>0],
                               fuel_Tank13[fuel_Tank13['Fuel Level Change']>0],
                               fuel_Tank14[fuel_Tank14['Fuel Level Change']>0],
                               fuel_Tank15[fuel_Tank15['Fuel Level Change']>0]],ignore_index=True)
fuel_order_loc1

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_order_loc1[fuel_order_loc1['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
fuel_order_loc2 = pd.concat([fuel_Tank16[fuel_Tank16['Fuel Level Change']>0],
                               fuel_Tank17[fuel_Tank17['Fuel Level Change']>0],
                               fuel_Tank18[fuel_Tank18['Fuel Level Change']>0],
                               fuel_Tank19[fuel_Tank19['Fuel Level Change']>0]],ignore_index=True)
fuel_order_loc2

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_order_loc2[fuel_order_loc2['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
fuel_order_loc3 = pd.concat([fuel_Tank20[fuel_Tank20['Fuel Level Change']>0],
                               fuel_Tank21[fuel_Tank21['Fuel Level Change']>0]],ignore_index=True)
fuel_order_loc3

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_order_loc3[fuel_order_loc3['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
fuel_order_loc4 = pd.concat([fuel_Tank22[fuel_Tank22['Fuel Level Change']>0],
                               fuel_Tank23[fuel_Tank23['Fuel Level Change']>0]],ignore_index=True)
fuel_order_loc4

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_order_loc4[fuel_order_loc4['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
fuel_order_loc5 = pd.concat([fuel_Tank24[fuel_Tank24['Fuel Level Change']>0],
                               fuel_Tank25[fuel_Tank25['Fuel Level Change']>0]],ignore_index=True)
fuel_order_loc5

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_order_loc5[fuel_order_loc5['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
fuel_order_loc6 = pd.concat([fuel_Tank26[fuel_Tank26['Fuel Level Change']>0],
                               fuel_Tank27[fuel_Tank27['Fuel Level Change']>0],
                                fuel_Tank28[fuel_Tank28['Fuel Level Change']>0]],ignore_index=True)
fuel_order_loc6

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_order_loc6[fuel_order_loc6['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
fuel_order_loc7 = pd.concat([fuel_Tank29[fuel_Tank29['Fuel Level Change']>0],
                               fuel_Tank30[fuel_Tank30['Fuel Level Change']>0]],ignore_index=True)
fuel_order_loc7

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_order_loc7[fuel_order_loc7['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
fuel_order_loc8 = pd.concat([fuel_Tank31[fuel_Tank31['Fuel Level Change']>0],
                               fuel_Tank32[fuel_Tank32['Fuel Level Change']>0]],ignore_index=True)
fuel_order_loc8

In [ ]:
pd.set_option('display.max_rows', None)
print(fuel_order_loc8[fuel_order_loc8['Fuel Level Change']>0]['Fuel Level Change'].value_counts().sort_index())
pd.set_option('display.max_rows', 60)

In [ ]:
combined_fuel_tank = pd.merge(fuel, tanks[['Tank ID', 'Tank Location']], on='Tank ID')
combined_fuel_tank

In [ ]:
combined_fuel_tank['Time stamp'] = pd.to_datetime(combined_fuel_tank['Time stamp'])
combined_fuel_tank['Date'] = combined_fuel_tank['Time stamp'].dt.date

daily_consumption = combined_fuel_tank.groupby(['Tank Location', 'Date']).agg({'Fuel Level': ['max', 'min']})
daily_consumption['Daily_Consumption'] = daily_consumption[('Fuel Level', 'max')] - daily_consumption[('Fuel Level', 'min')]
daily_consumption = daily_consumption['Daily_Consumption'].groupby('Tank Location').mean()

total_daily_consumption = daily_consumption.groupby('Tank Location').sum()

total_tank_capacity = tanks.groupby('Tank Location')['Tank Capacity'].sum()  

safety_stock = total_tank_capacity * 0.15 #Assuming safety stock is 15% of total capacity

total_daily_consumption, total_tank_capacity, safety_stock

## Cost savings

In [ ]:
def discount(a):
    '''Discounts vary with volume'''
    if a>=0 and a<=15000:
        return 0
    elif a<=25000:
        return 2
    elif a<=40000:
        return 3
    else:
        return 4

In [ ]:
## inflation dataframe
arr1=np.array([0.009,0.002,0.002,0.004,0.001,-0.001,0,0.001,0.002,0.001,0.003,-0.004])
infla=pd.DataFrame(arr1,columns=['inflation'])

In [ ]:
## purchase amount of each station
total_purchase=invoices.groupby('Invoice Gas Station Location')['Amount Purchased'].sum()
## purchase volume each time for each station
purchase_volume_eachtime=total_tank_capacity-safety_stock
## last purchase volume for each station
last_purchase=(total_purchase-total_tank_capacity)%purchase_volume_eachtime

In [ ]:
saving_list = []

# Calculate savings for each station
for i in range(1, 9):  
    saving = total_tank_capacity[i] * discount(total_tank_capacity[i]) + \
             (total_purchase[i] - total_tank_capacity[i]) // purchase_volume_eachtime[i] * purchase_volume_eachtime[i] * \
             discount(purchase_volume_eachtime[i]) + \
             last_purchase[i] * discount(last_purchase[i])
    saving_list.append(saving)

# Convert the list to a pd.Series with the specified index
saving_cost = pd.Series(saving_list, index=['Station1', 'Station2', 'Station3', 'Station4', 'Station5', 'Station6', 'Station7', 'Station8'])
saving_cost

## Ordering day

In [ ]:
## assign Weekday to Date
fuel['Date'] = pd.to_datetime(fuel['Time stamp'])
fuel['Weekday'] = fuel['Date'].dt.day_name()
fuel

In [ ]:
## find order day by boxplot
sns.set_style("darkgrid")
axes = sns.boxplot(data=fuel, x="Weekday", y="Fuel Level", palette="Set2")
axes.set(xlabel="Weekday",title="Different Fuel Level of Weekday")
plt.show()

the average fuel level from Sunday to Saturday are approximately the same, except for level of Monday is a little bit lower. The minimum value of Monday is the least one, and lower and higher quantile of Monday is nearly the least too. The maximum value of Monday is at a lower level. Thus we can consider ordering fuel on Monday.

## Increasing capacity

In [ ]:
## number of weeks from 2017-01-01 to 2019-08-15
d1=datetime.datetime(2017,1,1)
d2=datetime.datetime(2019,8,15)
days=(d2-d1).days
week_num=days/7

combined_fuel_tank['Time stamp'] = pd.to_datetime(combined_fuel_tank['Time stamp'])
combined_fuel_tank['Date'] = combined_fuel_tank['Time stamp'].dt.date
## calculating daily consumption for each tank
daily_consumption = combined_fuel_tank.groupby(['Tank Location','Tank ID', 'Date']).agg({'Fuel Level': ['max', 'min']})
daily_consumption['Daily_Consumption'] = daily_consumption[('Fuel Level', 'max')] - daily_consumption[('Fuel Level', 'min')]
## calculating average consumption for each station, each week
average_consumption=daily_consumption.groupby('Tank Location')[[('Daily_Consumption','')]].sum()/week_num

## find whether to increase capacity 
plt_1 = plt.plot(average_consumption.index,average_consumption , "red", label="Average weekly consumption per station")
plt_2 = plt.plot(total_tank_capacity.index, total_tank_capacity, "purple", label="Capacity for each station")
difference=total_tank_capacity-average_consumption[('Daily_Consumption','')]
plt_3 = plt.plot(difference.index,difference, "green", label="Difference")
ratio=difference/total_tank_capacity
plt_4 = plt.plot(ratio.index,ratio, "black", label="Ratio")

plt.title("Comparision between weekly consumption and capacity")
plt.xlabel("station")
plt.ylabel("amount")

plt.legend()

For all station, average weekly consumption amount is smaller than capacity, it's not that urgent to increase capacity of existing tanks.
If we only care about the difference between capacity and average consumption, then the difference for station 7 is the smallest one. Thus station 7 would benefit most from increasing capacity.
If we care about the ratio of difference to capacity, the ratio of each station seemd to be the same from the above chart. But if we put it in a separate chart like the following chart, we can find the ratio for station 1 is the lowest, namely the proportion of free capacity for station 1 is minimal, thus station 1 would benefit most from increasing capacity.

In [ ]:
difference=total_tank_capacity-average_consumption[('Daily_Consumption','')]
ratio=difference/total_tank_capacity
plt_1 = plt.plot(ratio.index,ratio, "black", label="Ratio")

plt.title("Comparision between weekly consumption and capacity")
plt.xlabel("station")
plt.ylabel("amount")

plt.legend()

## explore fuel type

In [ ]:
combined_fuel_tank['Time stamp'] = pd.to_datetime(combined_fuel_tank['Time stamp'])
combined_fuel_tank['Date'] = combined_fuel_tank['Time stamp'].dt.date
combined_fuel_tank1=combined_fuel_tank.merge(tanks, on ="Tank ID", how="inner")

In [ ]:
## consumption of fuel D
combined_fuel_tank1D=combined_fuel_tank1[combined_fuel_tank1['Tank Type']=='D']
## calculating daily consumption for each tank
daily_consumptionD = combined_fuel_tank1D.groupby(['Tank Location_x','Tank ID', 'Date']).agg({'Fuel Level': ['max', 'min']})
daily_consumptionD['Daily_Consumption'] = daily_consumptionD[('Fuel Level', 'max')] - daily_consumptionD[('Fuel Level', 'min')]
## calculating average consumption for each station
average_consumptionD=daily_consumptionD.groupby('Tank Location_x')[[('Daily_Consumption','')]].sum()

In [ ]:
## consumption of fuel U,P
combined_fuel_tank1G=combined_fuel_tank1[combined_fuel_tank1['Tank Type']!='D']
## calculating daily consumption for each tank
daily_consumptionG = combined_fuel_tank1G.groupby(['Tank Location_x','Tank ID', 'Date']).agg({'Fuel Level': ['max', 'min']})
daily_consumptionG['Daily_Consumption'] = daily_consumptionG[('Fuel Level', 'max')] - daily_consumptionG[('Fuel Level', 'min')]
## calculating average consumption for each station
average_consumptionG=daily_consumptionG.groupby('Tank Location_x')[[('Daily_Consumption','')]].sum()

In [ ]:
## purchase amount of fuel D in each station
invoicesd = invoices[invoices['Fuel Type']=='D']
invoicesd=invoicesd.groupby(['Invoice Gas Station Location'])['Amount Purchased'].sum()
invoicesd

In [ ]:
## purchase amount of fuel G in each station
invoicesg = invoices[invoices['Fuel Type']=='G']
invoicesg=invoicesg.groupby(['Invoice Gas Station Location'])['Amount Purchased'].sum()
invoicesg

In [ ]:
average_consumptionD

In [ ]:
## compare fuel D purchase amount and actual consumption
plt_1 = plt.plot(invoicesd.index,invoicesd , "red", label="purchase amount per station")
plt_2 = plt.plot(average_consumptionD.index, average_consumptionD, "purple", label="actual consumption per station")

difference=invoicesd-average_consumptionD['Daily_Consumption']
plt_3 = plt.plot(difference.index,difference, "green", label="Difference")


plt.title("compare fuel D purchase amount and actual consumption")
plt.xlabel("station")
plt.ylabel("amount")

plt.legend()

In [ ]:
## compare fuel G purchase amount and actual consumption
plt_1 = plt.plot(invoicesg.index,invoicesg , "red", label="purchase amount per station")
plt_2 = plt.plot(average_consumptionG.index, average_consumptionG, "purple", label="actual consumption per station")

difference=invoicesg-average_consumptionG['Daily_Consumption']
plt_3 = plt.plot(difference.index,difference, "green", label="Difference")


plt.title("compare fuel G purchase amount and actual consumption")
plt.xlabel("station")
plt.ylabel("amount")

plt.legend()